### **Deep Mutual Learning**

#### **Abstract**

- **不要 teacher-student，要 peer-to-peer。甚至效果更优**
- 数据集：CIFAR-100, Market-1501

---------------------------------------
#### **Introduction**
- 前有 explicit frugal architecture design [8], model compression [20], pruning [13], binarisation [18] and most interestingly model distillation [7].
  - **这都表明了小模型可以和大模型达到相同的能力**
- **真正的难点在于网络的优化，而不是网络的规模**
- **学习 teacher 的类概率或者特征表示**，可以比着传统的监督学习得到更多的信息，**这样也更容易**
- 本文提出 mutual learning，**每个student都有两个损失，一个是 conventional supervised learning loss，一个是 mimicry loss** that aligns each student's class posterior with the class probabilities of other students.
- **DML 优于 KD，并且并不需要一个 teacher model 就可以取得很好的效果**

- 当 peer-to-peer时，**在所有小型和未经训练的学生网络的情况下，额外的知识从哪来？**
    - DML与传统蒸馏方法一样，从次要数量中获取额外信息，但不同于传统蒸馏方法，它涉及学生之间的相互学习，**以汇集他们的共同估计**。
    - **它利用了学生网络对下一个最可能类的不同概率估计**。这些不同的概率估计为学生网络提供了额外的知识，这可以帮助它们收敛到更鲁棒的极小值，并在测试数据上具有更好的泛化性能。

- **为什么可以收敛到一个不错的解**，而不是 'the blind lead the blind'？
    - DML 通过**主要使用传统的监督学习损失**来引导每个学生，这意味着他们的个体表现通常会提高，不会盲目地跟随其他学生。
    - 在 DML 中，每个学生网络都被一个标准的监督学习损失函数引导。在监督学习中，所有网络很快会为每个训练实例预测相同的（正确）标签；**但是由于每个网络都从不同的初始条件开始，它们对下一个最可能类的概率估计会有所不同**。
    - 在 DML 中，学生群体有效地**汇总了它们对下一个最可能类的集体估计**。根据同伴的意见找出并匹配每个训练实例的其他最可能类**会增加每个学生的后验熵，这有助于它们收敛到更鲁棒（更平坦）的极小值**，并在测试数据上具有更好的泛化性能。

> **一言以蔽之，student 的初始值不同，后验熵被增加，所以 work**

- 网络越多，效果越好
- 可以混合大小异构网络
- 可以要一个 student，也可以 ensemble

--------------------------------------
#### **Related Work**
- **蒸馏本质上就是增加 后验熵和进行正则**
- Dual Learning
- **DML 的目的不是产生多样化的集成，而是让 student 之间尽可能的相似，这一点很难通过传统的监督学习完成**

------------------------------------------------
#### **Deep Mutual Learning**

![](../img/Figure-1_Deep-Mutual-Learning(DML)-schematic.png)


- 计算 $p_1$ 和 $p_2$ 时候**没有采用温度系数**
- $L_{C_i}$ 是和 ground truth 的交叉熵

> 具体损失函数见原文

![Algorithm 1: Deep Mutual Learning](../img/Algorithm_1-Deep-Mutual-Learning.png)

**Extension to Larger Student Cohorts 算法见原文**
- **唯一区别**就是计算 KL 散度的时候，是对所有的 peer 进行计算求和之后取平均
- **作者这里提出了一个替代方案，就是对于每个 student，将其所有的 peer 进行 averaged ensemble，然后再计算 KL 散度**
- **但是替代方案效果不如原始方案，原因在于丢失了高后验熵这一特性**

> 这种模型完全可以进行分布式训练，因为每个 student 都是独立的，只需要将其 peer 的输出传递过去就可以了

--------------------------------------------
#### **Experiments**

- dataset: CIFAR-100, Market-1501
- For evaluation, the **standard Cumulative Matching Characteristic (CMC) Rank-k accuracy and mean average precision (mAP)** metrics [27] are used.
- 代码使用 Tensorflow 实现，具体实现细节见原文

##### **Model Size**
![Table 1: Number of parameters on the CIFAR-100 dataset](../img/Table-1_Number-of-parameters-on-the-CIFAR-100-dataset.png)

##### **Results on CIFAR-100**
- DML 为设置为 2 个 student 的情况
- 大网络可以和小网络混合使用

![Table 2: Top-1 accuray(%) on the CIFAR-100 dataset](../img/Table-2_Top-1-accuracy-on-the-CIFAR-100-dataset.png)

##### **Results on Market-1501**
- DML 设置为 2 个 student 的情况

![Table 3: Comparative results on the Market-1501 dataset](../img/Table-3_Comparative-results-on-the-Market-1501-dataset.png)

##### **Comparison with Distillation**

![Table 4: Comparison with distillation on CIFAR-100(Top-1 accuracy(%)) and Market-1501 dataset(mAP(%))](../img/Table-4_Comparison-with-distillation-on-CIFAR-100_top-1-accuracy_and-Market-1501-dataset_mAP.png)

---------------------------------------------------------------
#### **Conclusion**
- 不要很强的 teacher，效果也可以挺好
- **蒸馏本质上就是增加 后验熵和进行正则**
- **DML 的目的不是产生多样化的集成，而是让 student 之间尽可能的相似，这一点很难通过传统的监督学习完成**
- 个人感觉缺陷还是有，因为模型不共享参数，导致模型太大

能不能共享重要的参数呢？